In [1]:
#Universidade Federal de Pernambuco
#Disciplina: Introdução a Microssimulação
#Professor: Jose Ricardo Nogueira
#Aluno: Paulo Francisco da Silva Junior

In [2]:
#importações
import pandas as pd
from numpy import nan

Calculdo da contribuição à previdência com base nas regras legais de 2019

In [3]:
#Importando banco
pnadc = pd.read_csv("/content/drive/MyDrive/Microsimulação/PNADC_2019_mine.csv",sep=",",decimal=".",dtype=float,na_values=" ")

In [4]:
#Corrigindo missings pra 0
pnadc.fillna(0,inplace=True)

Considerando que os valores dos salários primários e secundários presentes na PNADC estão mensalizados, o 13 salário e o adicional de férias serão divididos por caso a análise seja anualizada (o correto a se fazer).

In [6]:
#13 salário - trabalho principal
pnadc["SAL13_1"]=pnadc["V403312"]/12

In [7]:
#13 salário - trabalho secundário
pnadc["SAL13_2"]=pnadc["V405012"]/12

In [8]:
#Adicional de férias - trabalho principal
pnadc["AFERIAS1"]=(pnadc["V403312"]/3)

In [9]:
#Adicional de férias - trabalho secundário
pnadc["AFERIAS2"]=(pnadc["V405012"]/3)

In [10]:
#SIMULACAO DAS CONTRIBUICOES SOCIAIS (PNADC 2019)

In [11]:
#1. EMPREGADO E TRABALHADOR AVULSO 

In [12]:
#1.1 EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"]) <=1751.81),\
          "CONTE1"] = 0.08 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 1751.81) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=2919.72), \
          "CONTE1"] = 0.09 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 2919.72) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=5839.45), \
          "CONTE1"] = 0.11 * (pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==1) & \
          ((pnadc["V403312"] - pnadc["SALFAM1"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])> 5839.45), \
          "CONTE1"] = 0.11 * 5839.45


pnadc.fillna(value={"CONTE1":0},inplace=True)

In [13]:
#1.2 EMPREGADO E TRABALHADOR AVULSO (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=1751.81),
          "CONTE2"] = 0.08 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 1751.81) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=2919.72), \
          "CONTE2"] = 0.09 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 2919.72) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=5839.45), \
          "CONTE2"] = 0.11 * (pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==3) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] - pnadc["SALFAM2"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])> 5839.45), \
          "CONTE2"] = 0.11 * 5839.45

pnadc.fillna(value={"CONTE2":0},inplace=True)

In [14]:
#1.3 EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTE"]= pnadc["CONTE1"]+pnadc["CONTE2"]
pnadc.fillna(value={"CONTE":0},inplace=True)
#RECODE CONTE  (Lowest thru 0=0) 

In [15]:
#2. EMPREGADO DOMESTICO

In [16]:
#2.1 EMPREGADO DOMESTICO (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=1751.81), \
          "CONTED1"] = 0.08 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>1751.81) & \
          ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=2919.72), \
          "CONTED1"] = 0.09 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>2919.72) & \
          ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])<=5839.45), \
          "CONTED1"] = 0.11 *  (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])

pnadc.loc[(pnadc["VD4009"]==3) & ((pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])>5839.45), \
          "CONTED1"] = 0.11 * 5839.45 

pnadc.fillna(value={"CONTED1":0},inplace=True)

In [17]:
#2.2 EMPREGADO DOMESTICO (TRABALHO SECUNDARIO)
pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])<=1751.81), \
          "CONTED2"] = 0.08 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>1751.81) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"])<=2919.72), \
          "CONTED2"] = 0.09 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>2919.72) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"])<=5839.45), \
          "CONTED2"] = 0.11 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.loc[((pnadc["V4043"]==1) & (pnadc["V4048"]==1)) & \
          ((pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])>5839.45), \
          "CONTED2"] = 0.11 * 5839.45

pnadc.fillna(value={"CONTED2":0},inplace=True)

In [18]:
#2.3 EMPREGADO DOMESTICO (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTED"]= pnadc["CONTED1"]+pnadc["CONTED2"]
pnadc.fillna(value={"CONTED":0},inplace=True)

In [19]:
#4. SERVIDOR PUBLICO ESTATUTARIO FEDERAL

In [20]:
#4.1 SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL)
pnadc.loc[(pnadc["V4012"]==4) & (pnadc["V4028"]==1) & (pnadc["V4014"]==1), \
          "CONTFED1"] = 0.11 * (pnadc["V403312"] + pnadc["AFERIAS1"] + pnadc["SAL13_1"])
          
pnadc.fillna(value={"CONTFED1":0},inplace=True)

In [21]:
#4.2 SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO SECUNDARIO)
pnadc.loc[(pnadc["V4043"]==4) & (pnadc["V4047"]==1) & (pnadc["V4045"]==1), \
          "CONTFED2"] = 0.11 * (pnadc["V405012"] + pnadc["AFERIAS2"] + pnadc["SAL13_2"])

pnadc.fillna(value={"CONTFED2":0},inplace=True)

In [22]:
#4.3 SERVIDOR PÚBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL + SECUNDARIO)
pnadc["CONTFED"]= pnadc["CONTFED1"] + pnadc["CONTFED2"]
pnadc.fillna(value={"CONTFED":0},inplace=True)

In [23]:
#Utilizando os pesos, calculamos a contribuição total de cada grupo

In [24]:
#Valor total de contribuição - EMPREGADO E TRABALHADOR AVULSO (TRABALHO PRINCIPAL)
(pnadc["CONTE1"]*pnadc["V1032"]).sum()

13289773107.913937

In [25]:
#Valor total de contribuição - EMPREGADO E TRABALHADOR AVULSO (TRABALHO SECUNDARIO)
(pnadc["CONTE2"]*pnadc["V1032"]).sum()

64477134.11172675

In [26]:
#Valor total de contribuição - EMPREGADO DOMESTICO (TRABALHO PRINCIPAL)
(pnadc["CONTED1"]*pnadc["V1032"]).sum()

518110891.628599

In [27]:
#Valor total de contribuição - EMPREGADO DOMESTICO (TRABALHO SECUNDARIO)
(pnadc["CONTED2"]*pnadc["V1032"]).sum()

3363818.737039572

In [28]:
#Valor total de contribuição - SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO PRINCIPAL)
(pnadc["CONTFED1"]*pnadc["V1032"]).sum()

1863482249.7010322

In [29]:
#Valor total de contribuição - SERVIDOR PUBLICO ESTATUTARIO FEDERAL (TRABALHO SECUNDARIO)
(pnadc["CONTFED2"]*pnadc["V1032"]).sum()

2406514.148698481